# Reinforcement Learning - Q-Learning mit OpenAI Gym

Diese Übung wird mit einem Jupyter-Notebook angegeben, kann aber auch in Rmarkdown übertragen werden.

Ziel ist es, mit dem OpenAI Gym einen "eigenen" Agent zu erstellen.

Das Beispiel wird als "Selbstfahrendes" Taxi gezeigt - es kann aber auch jeder ein "eigenes" Gym ausprobieren.

## Beispiel: Selbstfahrendes Taxi:

Lasst uns eine Simulation von einem selbstfahrenden Taxi entwerfen. Das Hauptziel ist es, in einer vereinfachten Umgebung zu demonstrieren, wie man mit Hilfe von RL-Techniken einen effizienten und sicheren Ansatz zur Lösung dieses Problems entwickeln kann.

Die Aufgabe der Smartcab ist es, den Fahrgast an einem Ort abzuholen und an einem anderen Ort abzusetzen. Hier sind ein paar Dinge, um die wir uns mit unserem Smartcab gerne kümmern würden:

- Lasst den Beifahrer an der richtigen Stelle zurück.
- Zeitersparnis für die Fahrgäste durch minimale Zeitersparnis beim Absetzen der Fahrgäste
- Beachten Sie die Sicherheit der Fahrgäste und die Verkehrsregeln.

Es gibt verschiedene Aspekte, die hier bei der Modellierung einer RL-Lösung für dieses Problem berücksichtigt werden müssen: Belohnungen, Zustände und Aktionen.

# 1. Rewards
Da der Agent (der imaginäre Fahrer) belohnungsmotiviert ist und lernen wird, wie man das Taxi durch Trial-Erfahrungen in der Umwelt steuert, müssen wir die Belohnungen und/oder Strafen und deren Höhe entsprechend festlegen. Hier ein paar Punkte, die zu beachten sind:

- Der Agent sollte eine hohe positive Belohnung für einen erfolgreichen Dropoff erhalten, da dieses Verhalten sehr erwünscht ist.
- Der Agent sollte bestraft werden, wenn er versucht, einen Passagier an falschen Orten abzusetzen.
- Der Agent sollte eine leichte negative Belohnung dafür erhalten, dass er es nach jedem Zeitschritt nicht bis zum Ziel geschafft hat. "Leicht" negativ, weil wir es vorziehen würden, dass unser Agent zu spät kommt, anstatt falsche Schritte zu unternehmen und zu versuchen, das Ziel so schnell wie möglich zu erreichen.


# 2. State Space
Beim Reinforcement Learning begegnet der Agent einem Zustand und ergreift dann Maßnahmen entsprechend dem Zustand, in dem er sich befindet.

Der State Space ist die Gesamtheit aller möglichen Situationen, in denen unser Taxi leben könnte. Der Zustand sollte nützliche Informationen enthalten, die der Agent benötigt, um die richtigen Maßnahmen zu ergreifen.

Nehmen wir an, wir haben einen Trainingsbereich für unser Smartcab, in dem wir ihm beibringen, Menschen auf einem Parkplatz zu vier verschiedenen Orten (R, G, Y, B) zu transportieren:


![taxi_env](taxi_env.png)

Nehmen wir an, Smartcab ist das einzige Fahrzeug auf diesem Parkplatz. Wir können den Parkplatz in ein 5x5 Raster aufteilen, was uns 25 mögliche Taxistandorte gibt. Diese 25 Standorte sind ein Teil unseres Staatsraums. Beachten Sie, dass der aktuelle Standortzustand unseres Taxis koordiniert ist (3, 1).

Sie werden auch feststellen, dass es vier (4) Orte gibt, an denen wir einen Passagier abholen und absetzen können: R, G, Y, B oder ´[(0,0), (0,4), (4,0), (4,0), (4,3)]´ in Koordinaten (Reihe, Spalte). Unser illustrierter Passagier ist in Position Y und möchte zu Position **R** gehen.

Wenn wir auch einen (1) zusätzlichen Fahrgastzustand innerhalb des Taxis berücksichtigen, können wir alle Kombinationen von Fahrgast- und Zielorten nehmen, um zu einer Gesamtzahl von Zuständen für unsere Taxiumgebung zu gelangen; es gibt vier (4) Ziele und fünf (4 + 1) Fahrgastziele.

So hat unsere Taxiumgebung $5×5×5×5×4=500$ mögliche Zustände.

Der Agent trifft auf einen der 500 Zustände und er ergreift eine Aktion. Die Aktion in unserem Fall kann sein, sich in eine Richtung zu bewegen oder sich zu entscheiden, einen Fahrgast abzuholen oder abzusetzen.

## 3. Action Space 
Mit anderen Worten, wir haben sechs mögliche Aktionen:

1. `south`
2. `north` 
3. `east` 
4. `west` 
5. `pickup` 
6. `dropoff` 

Dies ist der Action Space: der Satz aller Aktionen, die unser Agent in einem bestimmten Zustand ausführen kann.

Sie werden in der obigen Abbildung feststellen, dass das Taxi in bestimmten Zuständen aufgrund von Mauern bestimmte Aktionen nicht ausführen kann. Im Code der Umgebung geben wir einfach eine -1 Strafe für jeden Mauerstoß und das Taxi bewegt sich nirgendwo hin. Dies wird nur Strafen mit sich bringen, die das Taxi dazu bringen, eine Umgehung der Mauer in Betracht zu ziehen.

# Setup der Umgebung

In [2]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Die Kern-Schnittstelle des Gym ist env, das ist die Unified Environment Interface. Nachfolgend sind die env-Methoden aufgeführt, die uns sehr hilfreich sein könnten:

- `env.reset`: Setzt die Umgebung zurück und gibt einen zufälligen Ausgangszustand zurück.
- `env.step(action)`: Schieben Sie die Umgebung um einen Zeitschritt nach vorne. Retouren
  - observation: Umweltbeobachtungen
  - reward: Ob Ihre Aktion nützlich war oder nicht.
  - done: Zeigt an, ob wir einen Passagier, auch eine Episode genannt, erfolgreich abgeholt und abgesetzt haben.
  - info: Zusätzliche Informationen wie Performance und Latenzzeiten für Debuggingzwecke
- `env.render`: Stellt einen Rahmen der Umgebung dar (hilfreich bei der Visualisierung der Umgebung).

In [3]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


## Ein State:

In [4]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
env.P[328] # Reward Table

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

## Lösung ohne Reinforcement Learning

In [6]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1871
Penalties incurred: 583


In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 409
State: 0
Action: 5
Reward: 20


# Aufgabe 1 : Implementierung von Q-Learning in python (5 Punkte)

**Tipps:**
Versuche mit einem Random-Agent-Template zu starten und wenn möglich - objektorientiert vorzugehen. (Random-Agent Klasse: https://github.com/openai/gym/blob/master/examples/agents/random_agent.py)

In [9]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Aufgabe 2: Vergleich von Q-Learning (10 Punkte)


Wir bewerten unsere Agenten anhand der folgenden Kennzahlen,

- **Durchschnittliche Anzahl der Strafen pro Episode**: Je kleiner die Zahl, desto besser die Leistung unseres Agenten. Im Idealfall möchten wir, dass diese Kennzahl Null oder sehr nahe bei Null liegt.
- **Durchschnittliche Anzahl der Zeitschritte pro Fahrt**: Wir wollen auch eine kleine Anzahl von Zeitschritten pro Episode, da wir möchten, dass unser Agent minimale Schritte (d.h. den kürzesten Weg) unternimmt, um das Ziel zu erreichen.
- **Durchschnittliche Belohnungen pro Zug**: Je größer die Belohnung, desto besser ist es, dass der Agent das Richtige tut. Deshalb ist die Entscheidung über die Belohnung ein wichtiger Teil des Verstärkungslernens. In unserem Fall, da sowohl Zeitschritte als auch Strafen negativ belohnt werden, würde eine höhere durchschnittliche Belohnung bedeuten, dass der Agent das Ziel so schnell wie möglich mit den geringsten Strafen erreicht".


Erstellen Sie eine Tabelle und vergleichen Sie Q-Learning mit dem "simplen" Ansatz von oben.


## Aufgabe 3: Hyperparameter (10 Punkte)

Die Werte von `alpha`, `gamma` und `epsilon` basierten hauptsächlich auf Intuition und etwas "Hit and Trial", aber es gibt bessere Möglichkeiten, gute Werte zu finden.

Im Idealfall sollten alle drei im Laufe der Zeit abnehmen, denn wenn der Agent weiter lernt, baut er tatsächlich widerstandsfähigere Vorgänger auf;

- *α*: (die Lernrate) sollte abnehmen, da Sie immer mehr an einer immer größeren Wissensbasis gewinnen.
- *γ*: Wenn Sie der Deadline immer näher kommen, sollte Ihre Präferenz für eine kurzfristige Belohnung steigen, da Sie nicht lange genug dabei sein werden, um die langfristige Belohnung zu erhalten, was bedeutet, dass Ihr Gamma sinken sollte.
- *ϵ*: Während wir unsere Strategie entwickeln, haben wir weniger Bedarf an Exploration und mehr Ausbeutung, um mehr Nutzen aus unserer Policy zu ziehen, so dass mit zunehmender Anzahl der Versuche epsilon abnehmen sollte.

Wie können die Hyperparameter "gesucht" werden?
Versuche mindestens eine Suchstrategie und gib die Hyperparameter an. Wie viele Iterationen benötigt der "minimale" Algorithmus?

In [ ]:
# your learning algorithm for tuning those alpha, gamma and epsilon